# Test 2b - Merge, Aggregate, Visualize

## Introduction

This problem tests your ability to merge, aggregate, and visualize data.
You'll work with realistic business data to answer questions about product profitability.

Before submitting, ensure your notebook:

- Includes your verification info (Banner ID and code) as the first cell
- Runs completely with "Kernel → Restart & Run All"
- Shows your work (don't just print final answers)
- Includes your interpretation responses at the end

## Dataset: Bean & Brew Coffee Shop

You've been hired to analyze sales data for Bean & Brew, a local coffee shop chain with three locations. The owner wants to understand which products are most profitable and how profitability varies across locations.

There are two data files:

- `sales.csv` - Individual transaction records
  - `transaction_id`: Unique transaction identifier
  - `timestamp`: Date and time of purchase (format: 'YYYY-MM-DD HH:MM:SS')
  - `location`: Store location ('Downtown', 'Campus', 'Suburb')
  - `product_id`: Product identifier (links to products table)
  - `quantity`: Number of units sold
  - `revenue`: Total revenue from transaction
  - `payment_method`: Payment type ('Cash', 'Credit', 'Mobile')
  - `loyalty_member`: Boolean indicating if customer is loyalty program member
- `products.csv` - Product catalog
  - `product_id`: Unique product identifier
  - `name`: Product name
  - `category`: Product category ('Coffee', 'Tea', 'Pastry', 'Sandwich')
  - `cost`: Cost to make/acquire one unit (for profit calculation)

The dataset contains two weeks of sales data (approximately 3,000 transactions) across 18 products. It includes both regular and premium items in each category.

We are interested in learning which product categories generate the most profit, and how does profitability vary across locations? Complete the following tasks to answer those questions and more.

## Scoring

Total: 30 points (+ 5 bonus)

- Tasks 1-6: 18 points
- Interpretation (Questions 1-4): 12 points
- Bonus: 5 points (optional)

## Generate Your Dataset

Before starting the analysis, you need to generate your unique dataset using your Banner ID.

1. Open a terminal in the folder containing `generate_test2b_data.py`
2. Activate your conda virtual environment
3. Run: `python generate_test2b_data.py`
4. Enter your 9-digit Banner ID when prompted (you'll enter it twice to confirm)

The script will create two files in the `data/` folder:

- `data/products.csv` - Product catalog
- `data/sales.csv` - Transaction records

Important: At the end, the script displays a verification block like this:

```
======================================================================
DATA VERIFICATION INFORMATION
======================================================================
banner id:          904123456
verification code:  5e0dXXXXXXXX
======================================================================
```

Copy the banner id and verification code lines and paste them into the cell below. This is required for submission.

```
banner id:          904290595
verification code:  9f0342ad12fa

```

## Setup: Load the Data

Run the cell below to load the data. The code is provided for you.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data (provided)
sales = pd.read_csv('data/sales.csv', parse_dates=['timestamp'])
products = pd.read_csv('data/products.csv')

print(f"Sales: {len(sales)} transactions")
print(f"Products: {len(products)} items in catalog")
print()
print("Sales preview:")
sales.head()

Sales: 2580 transactions
Products: 18 items in catalog

Sales preview:


,transaction_id,timestamp,location,product_id,quantity,revenue,payment_method,loyalty_member
0,1004,2025-11-07 06:23:00,Downtown,105,1.0,4.02,Mobile,False
1,1003,2025-11-07 06:27:00,Downtown,104,1.0,3.15,Credit,True
2,1005,2025-11-07 06:45:00,Campus,104,1.0,2.08,Mobile,False
3,1000,2025-11-07 06:47:00,Campus,104,1.0,2.28,Mobile,False
4,1002,2025-11-07 06:49:00,Suburb,109,1.0,1.65,Mobile,False


In [3]:
# Products catalog
products

,product_id,name,category,cost
0,101,Espresso,Coffee,1.2
1,102,Cappuccino,Coffee,1.8
2,103,Latte,Coffee,1.9
3,104,Americano,Coffee,1.0
4,105,Cold Brew,Coffee,1.5
5,106,Nitro Cold Brew,Coffee,2.5
6,107,Green Tea,Tea,0.8
7,108,Chai Latte,Tea,1.2
8,109,Herbal Tea,Tea,0.7
9,110,Premium Matcha,Tea,2.2


## Task 1: Data Quality Check

Investigate the sales data quality and coverage. Your code should answer the following questions:

1. How many missing values are in each column?
2. How many unique product IDs appear in the sales data? Compare this to the catalog size.
3. What is the date range (earliest and latest transaction)?

### Your Code

In [28]:
# Task 1: use isna(), nunique(), min/max on timestamp

sales_cols = sales.columns.tolist()

# Missing values
print("Missing Values in Sales Data")
print(sales[sales_cols].isna().sum())

# Missing Product IDs
unique_product_ids_sold = sales['product_id'].nunique()
unique_product_ids_in_catalog = products['product_id'].nunique()
print("\nNumber of Unique Products Sold:",unique_product_ids_sold)
print("Number of Unique Products in Catalog",unique_product_ids_in_catalog)

# Time Range
earliest_sale = sales['timestamp'].min()
latest_sale = sales['timestamp'].max()
print("\nSales Time Range:",earliest_sale,"-",latest_sale)


Missing Values in Sales Data
transaction_id    0
timestamp         0
location          0
product_id        0
quantity          5
revenue           2
payment_method    0
loyalty_member    0
dtype: int64

Number of Unique Products Sold: 21
Number of Unique Products in Catalog 18

Sales Time Range: 2025-11-07 06:23:00 - 2025-11-20 21:03:00


## Task 2: Merge Sales with Products

Merge the sales and products DataFrames to add product information (name, category, cost) to each transaction.

### Your Code

In [31]:
# Task 2: use pd.merge() with appropriate join type

# Merge sales and products, using left merge to keep all sales, even if
# the product ID doesn't exist in the products dataframe
merged = pd.merge(
    sales,
    products[['product_id', 'name', 'category', 'cost']],
    on='product_id',
    how='left'
)

merged.head()


,transaction_id,timestamp,location,product_id,quantity,revenue,payment_method,loyalty_member,name,category,cost
0,1004,2025-11-07 06:23:00,Downtown,105,1.0,4.02,Mobile,False,Cold Brew,Coffee,1.5
1,1003,2025-11-07 06:27:00,Downtown,104,1.0,3.15,Credit,True,Americano,Coffee,1.0
2,1005,2025-11-07 06:45:00,Campus,104,1.0,2.08,Mobile,False,Americano,Coffee,1.0
3,1000,2025-11-07 06:47:00,Campus,104,1.0,2.28,Mobile,False,Americano,Coffee,1.0
4,1002,2025-11-07 06:49:00,Suburb,109,1.0,1.65,Mobile,False,Herbal Tea,Tea,0.7


## Task 3: Verify the Merge

After merging, verify that the merge worked correctly:

1. Compare the shape of the merged DataFrame to the original sales data
2. Check if any rows have missing values in the product columns (name, category, cost)
3. If there are missing values, identify which product_id(s) are causing the problem

### Your Code

In [73]:
# Task 3: check shape, isna() on product columns, filter to find problem IDs

sales_shape = sales.shape
merged_shape = merged.shape

print("Sales Shape:",sales_shape)
print("Merged Shape:",merged_shape)

# Columns merged from the product dataframe with missing values
product_cols = ['name', 'category', 'cost']
product_cols_with_missing = merged[product_cols].isna().any().index.tolist()
print("Product Columns with Missing Data:",product_cols_with_missing)

# Problem IDs
problem_rows = merged[merged[product_cols_with_missing].isna().any(axis=1)]
problem_ids = problem_rows['product_id'].unique()
print("Missing Product IDs:", problem_ids)


Sales Shape: (2580, 8)
Merged Shape: (2580, 11)
Product Columns with Missing Data: ['name', 'category', 'cost']
Missing Product IDs: [201 200 199]


## Task 4: Profit by Category

First, calculate profit for each transaction: `profit = revenue - (cost × quantity)`

Then summarize profitability by product category. Your code should return a table that shows both total profit and number of transactions for each category. Sort it by total profit (highest first).

### Your Code

In [103]:
# Task 4: create profit column, then groupby with .agg()

merged['profit'] = merged['revenue'] - (merged['cost'] * merged['quantity'])

category_profit = (
    merged
    .groupby('category')
    .agg(
        total_profit=('profit', 'sum'),
        transactions=('transaction_id', 'count')
    )
    .sort_values('total_profit', ascending=False)
)

category_profit

,total_profit,transactions
category,,
Sandwich,3937.96,567
Coffee,2403.01,967
Pastry,1526.14,509
Tea,750.51,522


Analyze how profitability varies by both category and location. Your code should produce a table that shows the following for every combination of category and location:

- Total profit
- Average profit per transaction
- Number of transactions

Display the results as a flat table (i.e., without a nested/hierarchical index).

### Your Code

In [111]:
# Task 5: groupby multiple columns, .agg() with dict, reset_index()
grouped = (
    merged
    .groupby(['category', 'location'])
    .agg(
        total_profit=('profit', 'sum'),
        avg_profit=('profit', 'mean'),
        transactions=('transaction_id', 'count')  
    )
)

flat_group = grouped.reset_index()
flat_group

,category,location,total_profit,avg_profit,transactions
0,Coffee,Campus,1127.54,2.127434,530
1,Coffee,Downtown,1072.17,3.037309,355
2,Coffee,Suburb,203.30,2.479268,82
3,Pastry,Campus,550.00,2.534562,217
4,Pastry,Downtown,621.87,3.636667,171
5,Pastry,Suburb,354.27,2.927851,121
6,Sandwich,Campus,659.60,5.276800,125
7,Sandwich,Downtown,2351.89,7.892248,299
8,Sandwich,Suburb,926.47,6.524437,143
9,Tea,Campus,176.05,1.158224,153


## Task 6: Visualize Category Profitability

Create a bar chart showing total profit by product category. Use seaborn and ensure that the result:

- Has clearly labeled categories
- Includes a descriptive title
- Makes it clear which category is most/least profitable

### Your Code

In [ ]:
# Task 6: use sns.barplot() with aggregated data


## Interpretation

Concisely answer the following questions based on your analysis.


1. What might go wrong if you did not verify the merge in Task 3 before proceeding with the analysis?
2. In Tasks 4 and 5, you aggregated data at different levels of detail (by category, then by category and location). How did this change what you could learn from the data?
3. Reflect on the patterns you discovered in your analysis. What valuable insights did your work generate?
4. The coffee shop owner asks: "Based on your analysis, what's the single most important thing I should do to improve profitability?" Provide a specific, actionable recommendation. Justify your recommendation with evidence from your analysis (include specific numbers from your results). Also, suggest additional analysis that you feel might be worthwhile.

### Your Answers

*Interpretation responses here*

## Bonus: Time-Based Analysis (Optional)

For extra credit, investigate a time-based question of your choosing. Examples of questions you could explore include:

- Do sales patterns differ between weekdays and weekends?
- Which hours are busiest at each location?
- Is there a day of the week that stands out for profitability?

For full credit you must:

- State your question clearly
- Show your analysis code
- Summarize what you found in 2-3 sentences

In [ ]:
# Bonus: Your time-based analysis here


### Your Findings

*Bonus findings here (state your question and what you discovered)*

## Submission

Ensure your notebook:

- Has the verification block (Banner ID and code) as the first cell
- Runs completely from top to bottom without errors
- Shows all task outputs clearly
- Includes all interpretation responses
- Is saved and committed to your repository

Follow the submission instructions on Canvas to submit your work.